Digitize the XRF peaks table from https://xdb.lbl.gov/Section1/Table_1-2.pdf.

X-RAY DATA BOOKLET

Center for X-ray Optics and Advanced Light Source

Lawrence Berkeley National Laborator

In [3]:
import tabula
import pandas as pd
import numpy as np

pdf_path = "../files/XRF_periodic_table.pdf"
dfs = tabula.read_pdf(pdf_path, pages='all')

In [4]:
# tow dataframes don't have headers originally
# 1. I put the wrong header as the new last row
# 2. assign the missing headers by other dataframes
for i in [2, 4]:
    head_values = dfs[i].columns.tolist()
    head_values[-1] = np.nan
    dfs[i].loc[len(dfs[i])] = head_values
    dfs[i].columns = dfs[0].columns.tolist()
    print(dfs[i])

   Element        Kα1       Kα2       Kβ1       Lα1       Lα2       Lβ1  \
0   42  Mo  17,479.34  17,374.3  19,608.3  2,293.16  2,289.85  2,394.81   
1   43  Tc   18,367.1  18,250.8    20,619     2,424     2,420     2,538   
2   44  Ru   19,279.2  19,150.4  21,656.8  2,558.55  2,554.31  2,683.23   
3   45  Rh   20,216.1  20,073.7  22,723.6  2,696.74  2,692.05  2,834.41   
4   46  Pd   21,177.1  21,020.1  23,818.7  2,838.61  2,833.29  2,990.22   
5    47 Ag  22,162.92  21,990.3  24,942.4  2,984.31  2,978.21  3,150.94   
6   48  Cd   23,173.6  22,984.1  26,095.5  3,133.73  3,126.91  3,316.57   
7   49  In   24,209.7  24,002.0  27,275.9  3,286.94  3,279.29  3,487.21   
8   50  Sn   25,271.3  25,044.0  28,486.0  3,443.98  3,435.42  3,662.80   
9   51  Sb   26,359.1  26,110.8  29,725.6  3,604.72  3,595.32  3,843.57   
10  52  Te   27,472.3  27,201.7  30,995.7  3,769.33   3,758.8  4,029.58   
11   53  I   28,612.0  28,317.2  32,294.7  3,937.65  3,926.04  4,220.72   
12  54  Xe     29,779    

In [5]:
# concatenate all dataframes
ev_df = pd.DataFrame()

for df in dfs:
    ev_df = pd.concat([ev_df, df], axis=0)

ev_df

,Element,Kα1,Kα2,Kβ1,Lα1,Lα2,Lβ1,Lβ2,Lγ1,Mα1
0,3 Li,54.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4 Be,108.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5 B,183.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6 C,277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7 N,392.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9,92 U,"98,439","94,665","111,300","13,614.7","13,438.8","17,220.0","16,428.3","20,167.1","3,170.8"
10,93 Np,—,—,—,"13,944.1","13,759.7","17,750.2","16,840.0","20,784.8",—
11,94 Pu,—,—,—,"14,278.6","14,084.2","18,293.7","17,255.3","21,417.3",—
12,95 Am,—,—,—,"14,617.2","14,411.9","18,852.0","17,676.5","22,065.2",—


In [6]:
# convert the values to float
def convert_to_float(x):
    try:
        return float(x)
    except:
        if x == "—":
            return np.nan
        else:
            x = x.replace(",", "")
            return float(x)

In [7]:
# split the Element column into atomic number and element
zs = []
elements = []

for i in ev_df.Element:
    z, element = i.split()
    zs.append(int(z))
    elements.append(element)

In [8]:
tmp = ev_df.iloc[:, 1:].map(convert_to_float).reset_index(drop=True)

ev_compile_df = pd.concat(
    [pd.DataFrame({"Z": zs,"element": elements}), tmp],
    axis=1
)
ev_compile_df.sort_values("Z", axis=0, inplace=True)

# change the headers into names easier for typing
ev_compile_df.columns = ["Z", "element", "Ka1", "Ka2", "Kb1", "La1", "La2", "Lb1", "Lb2", "Ly1", "Ma1"]

ev_compile_df

,Z,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
0,3,Li,54.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Be,108.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,B,183.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,C,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,N,392.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
87,91,Pa,95868.0,92287.0,108427.0,13290.7,13122.2,16702.0,16024.0,19568.0,3082.3
88,92,U,98439.0,94665.0,111300.0,13614.7,13438.8,17220.0,16428.3,20167.1,3170.8
89,93,Np,NaN,NaN,NaN,13944.1,13759.7,17750.2,16840.0,20784.8,NaN
90,94,Pu,NaN,NaN,NaN,14278.6,14084.2,18293.7,17255.3,21417.3,NaN


In [11]:
#ev_compile_df.to_csv("../files/emission_peaks.csv")
ev_compile_df = pd.read_csv("../files/emission_peaks.csv", index_col=0)

In [10]:
ev_compile_df[ev_compile_df.element == "Rh"]

,Z,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
41,45,Rh,20216.1,20073.7,22723.6,2696.74,2692.05,2834.41,3001.3,3143.8,NaN


According to AWI's XRF core scanner manual P.14 ([URL](https://epic.awi.de/id/eprint/37355/4/XRFCore-Scanner_user-manualV2.pdf)), the conversion of channel is 20 eV/channel.

# Saliency map
## CaCO3

There are highlights at 

In [49]:
def filter_emission_peaks(df, min, max):
    mask = (df.iloc[:, 2:] > min*1000) & (df.iloc[:, 2:] < max*1000)
    tmp = df[mask].loc[mask.any(axis=1)].copy()
    tmp = tmp.replace(np.nan, "")
    element = df.loc[mask.any(axis=1), "element"].copy()
    return pd.concat([element, tmp.iloc[:, 2:]], axis=1)

In [50]:
# 2.2-2.6 kev
filter_emission_peaks(ev_compile_df, 2.2, 2.6)

,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
13,S,2307.84,2306.64,2464.04,,,,,,
37,Zr,,,,,,,2219.4,2302.7,
59,Nb,,,,,,2257.4,2367.0,2461.8,
38,Mo,,,,2293.16,2289.85,2394.81,2518.3,,
39,Tc,,,,2424.0,2420.0,2538.0,,,
40,Ru,,,,2558.55,2554.31,,,,
78,Tl,,,,,,,,,2270.6
79,Bi,,,,,,,,,2422.6


In [51]:
# 3.4-3.6 keV
filter_emission_peaks(ev_compile_df, 3.4, 3.6)

,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
16,K,,,3589.6,,,,,,
43,Ag,,,,,,,,3519.59,
44,Cd,,,,,,,3528.12,,
45,In,,,,,,3487.21,,,
46,Sn,,,,3443.98,3435.42,,,,
47,Sb,,,,,3595.32,,,,


In [55]:
filter_emission_peaks(ev_compile_df, 9.5, 10.5)

,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
27,Zn,,,9572.0,,,,,,
28,Ga,,,10264.2,,,,,,
29,Ge,9886.42,9855.32,,,,,,,
67,Yb,,,,,,,,9780.1,
68,Lu,,,,,,,,10143.4,
70,Ta,,,,,,,9651.8,,
71,W,,,,,,9672.35,9961.5,,
72,Re,,,,,,10010.0,10275.2,,
73,Os,,,,,,10355.3,,,
76,Au,,,,9713.3,9628.0,,,,


In [53]:
# 13.4-13.45 keV
filter_emission_peaks(ev_compile_df, 13.4, 13.45)

,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
80,Po,,,,,,13447.0,,,
88,U,,,,,13438.8,,,,


## TOC
Mostly it has similar peaks as CaCO3, but with one additional peak around 17.5.

In [54]:
# 17.4-17.6 keV
filter_emission_peaks(ev_compile_df, 17.4, 17.6)

,element,Ka1,Ka2,Kb1,La1,La2,Lb1,Lb2,Ly1,Ma1
38,Mo,17479.34,,,,,,,,
